In [1]:
import requests
import csv
import json
import re
import ast

In [13]:
with open('../tempData/en2tw.txt', 'rt', encoding='utf-8', newline='') as fin:
    cin = csv.reader(fin, delimiter='	')
    en2twList = [row for row in cin]
print(len(en2twList))

for en, tw in en2twList:
    en2tw[en] = tw
    
def en2twFn(en):
    if en in en2tw:
        return en2tw[en]
    return en

808


In [3]:
url = 'https://nookplaza.net/main.dart.js?v=9'
r = requests.get(url)
mainJs = r.text

In [4]:
# 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)
print(len(m))

m = [item.encode("utf-8").decode('unicode-escape') for item in m]
m = [ast.literal_eval(item) for item in m if not 'null' in item]
print(len(m))

nameMap = {}
for item in m:
    nameMap[item["en-us"].lower()] = item

5715
5713


In [19]:
nameMap['Black']

KeyError: 'Black'

In [6]:
url = 'https://api.nookplaza.net/items?category=Furniture'
r = requests.get(url)
data = r.json()['results']

In [20]:
idMap = {}
dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'bodys', 'pattrens', 'variations']

output = [['id', '名稱(英文)', '名稱(日文)', '名稱(中文)', '買價', '賣價', '分類', '標籤', '主題', '可互動', '大小', '取得方式', 'DIY', '圖片', '款式說明', '樣式說明', '款式改造', '樣式改造', '款式', '樣式', '檔名對照索引']]
for item in data:
    if item["category"] == "Art":
        continue
    name = nameMap[item['name'].lower()]
    nameMap[item['name'].lower()]['id'] = item['content']['internalID']
    idMap[item['content']['internalID']] = item

#     print(item['content'])
    item['content']['category_'] = en2twFn(item['content']['category'])
    item['content']['tag_'] = en2twFn(item['content']['tag'])
    item['content']['themes_'] = [en2twFn(theme) for theme in item['content']['themes']]
    item['content']['obtainedFrom_'] = [en2twFn(item) for item in item['content']['obtainedFrom']]
    
    # 名稱 買價 賣價
    row = [int(item['content']['internalID']), name['en-us'], name['ja-jp'], name['zh-tw'], item['content']['buy'], item['content']['sell']]

    # 分類 標籤 主題
    # 'category' 'tag' 'themes'
    if "沒有" in item['content']['themes_']:
        item['content']['themes_'].remove("沒有")
    row += [item['content']['category_'], item['content']['tag_'], '、'.join(item['content']['themes_'])]
    
    # 互動 大小
    row += [item['content']['interact'], '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])]

    # 取得方式 DIY
    if not 'dIY' in item['content']:
        print(item)
    row += ['、'.join(item['content']['obtainedFrom_']), item['content']['dIY']]

    # 圖片檔名
    row += [item['content']['filename']]
    
    # 改造-款式 樣式
    # bodyCustomize bodyColor bodyTitle; patternCustomize patternColor patternTitle
    row += [item['content']['bodyTitle'], item['content']['patternTitle'], item['content']['bodyCustomize'], item['content']['patternCustomize']]

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        filename = content['filename']
        bodyIndex, pattrenIndex = filename.split('_')[-2:]
        
        bodyColor = '' if content['bodyColor'] is None else content['bodyColor']
        patternColor = '' if content['patternColor'] is None else content['patternColor']
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex

#         response = requests.get(content['image'])
#         with open('../img/furniture/{}.png'.format(filename), "wb") as fout:
#             fout.write(response.content)
        
    row += ['、'.join(list(variations['bodys'].keys())), '、'.join(list(variations['pattrens'].keys())), variations]
       
#     print(row)
    output.append(row)
output[1:] = sorted(output[1:], key=lambda k: k[0])

In [22]:
with open('items.csv', 'wt', encoding='utf-8', newline='') as fout:
    csvout = csv.writer(fout)
    csvout.writerows(output)

jsonOutput = []
for row in output[0:]:
    j = {}
    for index, item in enumerate(row):
        j[dictName[index]] = item
    jsonOutput.append(j)

with open('../../nook_link/src/data/furniture.json', 'wt') as fout:
    fout.write(json.dumps(jsonOutput))
    
with open('furniture.json', 'wt') as fout:
    fout.write(json.dumps(jsonOutput))